In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
import gc

from pyproj import Proj, transform, Transformer
import geopandas
from geopy.geocoders import Nominatim
import pgeocode

In [ ]:
%matplotlib inline

In [ ]:
path = r'C:\Users\nukis\Documents\Projects\08. Bike Safety'

In [ ]:
df_crash = pd.read_csv(os.path.join(path, '01. Data', 'Original data', '2021_DATA_SA_Crash.csv'), low_memory=False)

In [ ]:
df_un = pd.read_csv(os.path.join(path, '01. Data', 'Original data', '2021_DATA_SA_Units.csv'), low_memory=False)

In [ ]:
df_cas = pd.read_csv(os.path.join(path, '01. Data', 'Original data', '2021_DATA_SA_Casualty.csv'), low_memory=False)

In [ ]:
# Command to maximize view of rows and columns
pd.options.display.max_rows = None
pd.options.display.max_columns = len(df_crash.columns)

In [ ]:
df_crash = df_crash.reset_index()
df_crash.drop('index', axis=1, inplace=True)
df_crash.head() 

In [ ]:
df_crash.shape

# Data Cleaning

In [ ]:
# Check for missing values

df_crash.isnull().sum()

In [ ]:
# Check for missing values

pd.DataFrame(data = [round(i/len(df_crash) * 100, 2) for i in df_crash.isnull().sum().to_list()], index = df_crash.columns, columns = ['Missing Values %']).T

In [ ]:
# Check for duplicates

dups = df_crash.duplicated()
dups.sum() #No dups

In [ ]:
# Check for mixed-type data in dataframe

for col in df_crash.columns.tolist():
  weird = (df_crash[[col]].applymap(type) != df_crash[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_crash[weird]) > 0:
    print (col) # No mixed-type

In [ ]:
df_crash['LGA Name'].value_counts(dropna=False)

In [ ]:
df_crash['DUI Involved'].value_counts(dropna=False)

In [ ]:
df_crash['Drugs Involved'].value_counts(dropna=False)

Looking at the data above, following steps are performed:
1. Suburb and LGA Name will converted to title format for easy reading.
2. LGA NAN will be filled using X, Y. Strip '.' from LGA NAme.
2. DUI Involved and Drugs Involved which is not 'Y' will be filled with '0' and 'Y' will be converted to '1'.
3. Convert Year and Postcode to object.
4. Convert Time format. Drop unncessary columns.
5. Fill ACCLOC_X and ACCLOC_Y NAN and retrieve find lat and long features.

In [ ]:
# Step 1

df_crash['Suburb'] = df_crash['Suburb'].str.title()
df_crash['LGA Name'] = df_crash['LGA Name'].str.title()

In [ ]:
# Step 2
# Extracting lat long data

source_crs = 3107 # Coordinate system of the file for Australia
target_crs = 4326 # Global lat-lon coordinate system

xy_to_latlon = Transformer.from_crs(source_crs, target_crs)

geolocator = Nominatim(user_agent = 'geoapiExercises')

df_crash['LGA Name'].fillna(0, inplace=True)

for i in range(len(df_crash)):  
    if df_crash.iloc[i, 4] == 0:
        try:
            x = df_crash.iloc[i, -3]
            y = df_crash.iloc[i, -2]

            lat, lon = xy_to_latlon.transform(x, y)
            lat = str(lat)
            lon = str(lon)

            location = geolocator.reverse(lat + ','+ lon)
            df_crash.iloc[i, 4] = location.raw['address']['city_district']
        except:
            df_crash.iloc[i, 4] = location.raw['address']['county']
            print(i)

In [ ]:
df_crash['LGA Name'] = df_crash['LGA Name'].apply(lambda x: ' '.join(x.split()).replace('.', ''))

In [ ]:
# Step 3

df_crash['DUI Involved'].fillna(0, inplace = True)
df_crash.loc[df_crash['DUI Involved'] == 'Y', 'DUI Involved'] = 1

df_crash['Drugs Involved'].fillna(0, inplace = True)
df_crash.loc[df_crash['Drugs Involved'] == 'Y', 'Drugs Involved'] = 1

In [ ]:
# Step 4

df_crash[['Year', 'Postcode']] = df_crash[['Year', 'Postcode']].astype('str')

In [ ]:
# Step 5

def convert24(str1):

    # Checking if last two elements of time
    # is AM and first two elements are 12
    if str1[-2:] == "am" and str1[:2] == "12":
        return "00" + str1[2:-2]
         
    # remove the AM    
    elif str1[-2:] == "am":
        return str1[:-2]
     
    # Checking if last two elements of time
    # is PM and first two elements are 12
    elif str1[-2:] == "pm" and str1[:2] == "12":
        return str1[:-2]
         
    else:
         
        # add 12 to hours and remove PM
        return str(int(str1[:2]) + 12) + str1[2:5]

In [ ]:
# Driver Code 
time_convert = []

for i in df_crash['Time']:
    time_convert.append(convert24(i))
    
df_crash['Hour'] = time_convert 

In [ ]:
df_crash = df_crash.drop(columns = ['Time', 'UNIQUE_LOC'], axis=1)
df_crash = df_crash.dropna(subset=['ACCLOC_X', 'ACCLOC_Y'], how='all')

In [ ]:
# Step 5

df_crash['Lat'] = ''
df_crash['lon'] = ''

In [ ]:
# Step 5

def address_concate(data):
    address = data['Suburb'] + ', ' + data['Postcode']
    return address

In [ ]:
def latlon(x, y):
    source_crs = 3107 # Coordinate system of the file for Australia
    target_crs = 4326 # Global lat-lon coordinate system

    xy_to_latlon = Transformer.from_crs(source_crs, target_crs)
    
    lat, lon = xy_to_latlon.transform(x, y)
    
    lat = str(lat)
    lon = str(lon)
    
    location = geolocator.reverse(lat + ','+ lon)
    
    lat_result = location.raw['lat']
    lon_result = location.raw['lon']
    
    return (lat_result, lon_result)

In [ ]:
def postcode(postcode):
    nomi = pgeocode.Nominatim('au')
    result = nomi.query_postal_code(postcode)

    lat_result = result['latitude']
    lon_result = result['longitude']
    
    return (lat_result, lon_result)

In [ ]:
# Step 5
# Extracting Lat and Lon from X, Y

lat_result = []
lon_result = []

for i in range(len(df_crash)):
    try:
        lat, lon = latlon(df_crash.iloc[i, 29], df_crash.iloc[i, 30]) # 29 & 30 are X and Y
        
        lat_result.append(lat)
        lon_result.append(lon)
    except:
        lat, lon = postcode(df_crash.iloc[i, 3]) # 3 is postcode
        
        lat_result.append(lat)
        lon_result.append(lon)

In [ ]:
df_crash['Lat'] = lat_result
df_crash['lon'] = lon_result

In [ ]:
df_crash.rename(columns={'lon': 'Lon'}, inplace=True)
df_crash = df_crash.drop(columns = ['Year', 'Other Feat', 'Unit Resp', 'ACCLOC_X', 'ACCLOC_Y'], axis=1)

In [ ]:
df_crash.head()

In [ ]:
# Export data to pkl

df_crash.to_pickle(os.path.join(path, '01. Data', 'Prepared data', 'crash_cleaned.pkl'))
df_crash.to_csv(os.path.join(path, '01. Data', 'Prepared data', 'crash_cleaned.csv'), sep = ',')

In [ ]:
gc.collect()